In [ ]:
# Tensorflowが使うCPUの数を制限します。(VMを使う場合)
%env OMP_NUM_THREADS=1
%env TF_NUM_INTEROP_THREADS=1
%env TF_NUM_INTRAOP_THREADS=1

from tensorflow.config import threading
num_threads = 1
threading.set_inter_op_parallelism_threads(num_threads)
threading.set_intra_op_parallelism_threads(num_threads)

#ライブラリのインポート
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import utils

## ミニバッチ法のKerasによる実装
Tensorflow/KerasではMinibatch法が簡単に使えるようようになっています。
基礎編で使った2次元データとMLPモデルを例に実験します。

まずは一度に全てのデータを使ってロスの計算を行うバッチ学習を試してみます。

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense

# データ点の取得 (データセットのサイズは1100です。)
x, t = utils.dataset_for_mlp()

# モデルの定義
model = Sequential(
    [
        Input(shape=(2,)),
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=1, activation="sigmoid"),  # ノード数が1の層を追加。活性化関数はシグモイド関数。
    ]
)

#  誤差関数としてクロスエントロピーを指定。最適化手法はadam
model.compile(loss="binary_crossentropy", optimizer="SGD")

# バッチ法によるトレーニング
_ = model.fit(
    x=x,
    y=t,
    batch_size=len(x),  # バッチサイズ。一回のステップで全データ(1100行のデータ)を使うようにする。
    epochs=1,  # 学習のステップ数
    verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

次に1行ずつロスを計算するオンライン学習で学習を実行します。

In [ ]:
# モデルの定義
model = Sequential(
    [
        Input(shape=(2,)),
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=1, activation="sigmoid"),  # ノード数が1の層を追加。活性化関数はシグモイド関数。
    ]
)

#  誤差関数としてクロスエントロピーを指定。最適化手法はadam
model.compile(loss="binary_crossentropy", optimizer="SGD")

# オンライン法によるトレーニング
_ = model.fit(
    x=x,
    y=t,
    batch_size=1,  # バッチサイズ。一回のステップで1行のデータのみを使うようにする。
    epochs=1,  # 学習のステップ数
    verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

Kerasの出力するログを見比べるといくつか値が変わっています。

- 左端の"1/1", "1100/1100"は重みの更新のステップ数を表しています。バッチ法では全データを一度に使って重みの更新をしているため、1エポックあたりの重みの更新回数は1回です。一方で、オンライン学習は1行ごとに更新を行うため、合計1100回重みの更新がおこなわれます。
- 中程にある "XXms/step"は1回の重み更新にかかる時間を表しています。バッチ法では全データを使うため、1回の重みの更新に時間がかかります。オンライン学習では1行のみなので、1回の重み更新計算は高速です。
- その左の "XXs"はトータルでかかった時間を表します。オンライン学習では1ステップあたりの時間は短いですが、1100回重みの更新をする必要があるためトータルの時間はバッチ法よりも長くかかっています。
- 今回は全データを１回ずつ使用して(エポック数1で)学習したためバッチ法では学習が十分に進んでいません。エポック数(`fit`メソッドの中の`epochs`)を増やすことで学習をさらに進めることができます。

次にミニバッチ法を試してみましょう。バッチサイズは10としてみます。

In [ ]:
# モデルの定義
model = Sequential(
    [
        Input(shape=(2,)),
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=64, activation="relu"),  # ノード数が64の層を追加。活性化関数はReLU。
        Dense(units=1, activation="sigmoid"),  # ノード数が1の層を追加。活性化関数はシグモイド関数。
    ]
)

#  誤差関数としてクロスエントロピーを指定。最適化手法はadam
model.compile(loss="binary_crossentropy", optimizer="SGD")

# オンライン法によるトレーニング
_ = model.fit(
    x=x,
    y=t,
    batch_size=10,  # バッチサイズ。一回のステップで10行のデータのみを使うようにする。
    epochs=1,  # 学習のステップ数
    verbose=1,  # 1とするとステップ毎に誤差関数の値などが表示される
)

# Figureの作成 (キャンバスの作成)
fig, ax = plt.subplots()

# パーセプトロンの出力を等高線プロット
utils.plot_prediction(model, ax=ax)

# データ点をプロット
utils.plot_datapoint(x, t, ax=ax)

# 図を表示
plt.show()

1回の重み更新で10イベントを処理するので、全体として110ステップ分処理することになりました。
振る舞いはバッチ学習とオンライン学習の中間くらいになっています。

今回はデータセットがシンプルだったため、オンライン学習でも問題なく学習が進みましたが、オンライン学習は学習が不安定になることが多いです。
そのため実際の深層学習モデル学習の際は、バッチサイズが32 ~ 2048程度のミニバッチ学習を使うことが多いです。